# **Feature Engineering - credit_card_balance**

### **Importar Bibliotecas**

In [1]:
# !/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701732194661_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'credit_card_balance'.**

In [3]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1701732194661_0001/container_1701732194661_0001_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [4]:
file='credit_card_balance.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output = 'abt_credit_card_balance'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/credit_card_balance.csv

In [8]:
credit_card_balance_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
credit_card_balance_00.registerTempTable("credit_card_balance_00")
credit_card_balance_00.cache()
qtd=credit_card_balance_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 3840312
/mnt1/yarn/usercache/livy/appcache/application_1701732194661_0001/container_1701732194661_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.

In [9]:
credit_card_balance_00.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

In [10]:
credit_card_balance_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_PREV: string (nullable = true)
 |-- SK_ID_CURR: string (nullable = true)
 |-- MONTHS_BALANCE: string (nullable = true)
 |-- AMT_BALANCE: string (nullable = true)
 |-- AMT_CREDIT_LIMIT_ACTUAL: string (nullable = true)
 |-- AMT_DRAWINGS_ATM_CURRENT: string (nullable = true)
 |-- AMT_DRAWINGS_CURRENT: string (nullable = true)
 |-- AMT_DRAWINGS_OTHER_CURRENT: string (nullable = true)
 |-- AMT_DRAWINGS_POS_CURRENT: string (nullable = true)
 |-- AMT_INST_MIN_REGULARITY: string (nullable = true)
 |-- AMT_PAYMENT_CURRENT: string (nullable = true)
 |-- AMT_PAYMENT_TOTAL_CURRENT: string (nullable = true)
 |-- AMT_RECEIVABLE_PRINCIPAL: string (nullable = true)
 |-- AMT_RECIVABLE: string (nullable = true)
 |-- AMT_TOTAL_RECEIVABLE: string (nullable = true)
 |-- CNT_DRAWINGS_ATM_CURRENT: string (nullable = true)
 |-- CNT_DRAWINGS_CURRENT: string (nullable = true)
 |-- CNT_DRAWINGS_OTHER_CURRENT: string (nullable = true)
 |-- CNT_DRAWINGS_POS_CURRENT: string (nullable = true)
 |-- CN

### **Comentário**

- Cada "SK_ID_PREV" se refere a um único "SK_ID_CURR".

- Um "SK_ID_CURR" pode corresponder a diferentes "SK_ID_PREV".

- Ou seja, podemos referenciar as variáveis criadas com a tabela "application", assim como, a tabela "previous_application".

In [11]:
# credit_card_balance_00 = spark.sql('''
# SELECT
#   *
# FROM
#   credit_card_balance_00
# WHERE
#   SK_ID_CURR = '405549'
# ''')

# credit_card_balance_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# # Contar valores distintos da variável SK_ID_CURR
# qtde_distinct_sk_id_curr = spark.sql('''
# SELECT
#   COUNT(DISTINCT SK_ID_CURR) AS qtde_distinct_sk_id_curr
# FROM
#   credit_card_balance_00
# ''')

# # Exibir a quantidade de valores distintos
# qtde_distinct_sk_id_curr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# **Criação de variáveis preditivas**

## **credit_card_balance**

In [13]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['NAME_CONTRACT_STATUS']
var_numericas = ['MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', \
                 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', \
                 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT',\
                 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE',\
                 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',\
                 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', \
                 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF']

# Definir flags de meses
meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
credit_card_balance_00.createOrReplaceTempView('credit_card_balance_00')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando flags de janela para histórico:**

- últimos 3 meses

- últimos 6 meses

- últimos 9 meses

- últimos 12 meses

In [15]:
credit_card_balance_01 = spark.sql("""
SELECT
    *,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 3 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_3_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_6_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_9_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) - 12 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_CURR`) THEN 1
        ELSE 0
    END AS ultimos_12_meses_flag
FROM credit_card_balance_00
ORDER BY `SK_ID_CURR`, MONTHS_BALANCE;
""")

credit_card_balance_01.createOrReplaceTempView("credit_card_balance_01")
credit_card_balance_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3840312

In [16]:
# credit_card_balance_01.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [17]:
# Importar SparkSession
from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

df_original_spark = credit_card_balance_01

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        # Filtrar valores distintos não nulos
        distinct_values = df_original_spark.select(var_cat).distinct().na.drop(subset=[var_cat]).rdd.flatMap(lambda x: x).collect()

        for dom_cat_value in distinct_values:
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower()}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

                for mes_flag in meses_flag:
                    resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower()}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_')},"

                    # Adicionar o resultado à lista
                    resultados.append(resultado)

                    # Incrementar o contador
                    contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 2240

In [18]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
    # print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [19]:
credit_card_balance_02 = spark.sql(f'''
SELECT
  SK_ID_CURR as SK_ID_CURR_credit_card_balance,
  {resultados_str}
FROM
  credit_card_balance_01
GROUP BY
  SK_ID_CURR
''')

credit_card_balance_02.createOrReplaceTempView("credit_card_balance_02")
credit_card_balance_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

103558

In [20]:
print(len(credit_card_balance_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2801

In [ ]:
# credit_card_balance_02.show(5, truncate=False)

## **Salvando tabela no diretório do S3.**

In [22]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)
credit_card_balance_02.coalesce(1).write.\
parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_credit_card_balance

In [ ]:
# abt_credit_card_balance=sqlContext.read.format("parquet").load('s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_credit_card_balance')
# abt_credit_card_balance.registerTempTable("abt_credit_card_balance")

In [ ]:
# abt_credit_card_balance.show(5,truncate=False)